# Connecting to BigQuery
## resource with problem statement of this demo
- https://youtu.be/ieaqfU1BwJ8
## create queries to filter data, extract features, and clean where possible

### below is an **example** query
- here a query is made and stored in dataframe
- more tags, and api documentation are provided here: https://googleapis.dev/python/bigquery/latest/magics.html



In [1]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


### example query to strip **html** , and replace "|" with "," for delimiting

In [303]:
%%bigquery dataframe --verbose
SELECT 
LOWER(CONCAT(title, " ", REGEXP_REPLACE(body, "<[^>]*>", ""))) as article,
REPLACE(tags, "|", ",") AS  tags
FROM `bigquery-public-data.stackoverflow.posts_questions`
WHERE REGEXP_CONTAINS(tags, r"(?:python|javascript|mobil|react|pandas)")
limit 30000

Executing query with job ID: 813fa0cf-9a73-40e6-93f1-9f8a0b4b33e6
Query executing: 0.29s
Query complete after 0.52s


Downloading: 100%|██████████| 30000/30000 [00:01<00:00, 20131.86rows/s]


### save query to python pandas dataframe
- more data manipulation and rangling can be done in python if needed

In [7]:
type(dataframe)

pandas.core.frame.DataFrame

In [304]:
dataframe

,article,tags
0,if not opera mini this question is quite simpl...,"javascript,jquery,html,opera,opera-mini"
1,store blob data with django and sqlite first o...,"python,django,orm,blob,blobstorage"
2,classes and functions in js? here is some samp...,"javascript,class"
3,serving media in django i was going through se...,"python,django,django-templates"
4,which for loop is more efficient in javascript...,"javascript,for-loop,comparison,performance"
...,...,...
29995,dynamic page (jquery) not loading javascript c...,"javascript,jquery,html,css"
29996,jquery show div on hover in php cicle from dat...,"javascript,jquery,html,css"
29997,avoid text wrapping while doing a jquery anima...,"javascript,jquery,html,css"
29998,zooming photo using jquery hi i'm trying to zo...,"javascript,jquery,html,css"


### python string formating -- **data preperation for model training**
- code below removes all "," and new lines in article
- then the tags are filtered and delimited by ","
- this is done because training **csv** file is delimited by ","

In [324]:
#creating deep copy to test with
test_dataframe = dataframe.copy(deep=True)

# items selected for training
interest_items = ["python", "reactjs", "pandas", "numpy", "javascript","sql", "jquery", "django", "php"]
with open("file_write.csv", "w") as file_write:
    for index, row in test_dataframe.iterrows():

        #csv format requires delimiting by comma, appending comma at end of question
        test_dataframe.at[index, 'article'] = test_dataframe.at[index, 'article'].replace(',','')
        test_dataframe.at[index, 'article'] = test_dataframe.at[index, 'article'].replace('\n','')
        test_dataframe.at[index, 'article'] = test_dataframe.at[index, 'article'] + ','

        clean_list = ""
        #create iterable list of tags
        row['tags'] = row['tags'].split(',') 
        for tag in row['tags']:
            if tag in interest_items:
                clean_list += tag + "," 
        # # delete rows which don't contain interest items
        if not clean_list:
            test_dataframe.drop(index, inplace=True)

        else:
            clean_list = clean_list[:-1]
            test_dataframe.at[index, 'tags'] = clean_list
            # writing datafram with filtered valuews into comma delimited csv
            file_write.write(test_dataframe.at[index, 'article'] + clean_list + "\n")
        

### note the difference between original and filtered dataframe
- test_dataframe == filtered
- dataframe == original
- test_dataframe even has dropped certain rows, due to a lack of matched interest items

In [331]:
test_dataframe.head()

,article,tags
0,if not opera mini this question is quite simpl...,"javascript,jquery"
1,store blob data with django and sqlite first o...,"python,django"
2,classes and functions in js? here is some samp...,javascript
3,serving media in django i was going through se...,"python,django"
4,which for loop is more efficient in javascript...,javascript
5,parameters in ajax php callback function i am ...,"php,javascript,jquery"
6,can one author a library api – one accessible ...,python
7,auto convert dictionary into list of dictionar...,python
8,complex numbers and fractals i am trying to ad...,python
10,is there any better way to handle list of coun...,javascript


In [314]:
dataframe.head()

,article,tags
0,if not opera mini this question is quite simpl...,"javascript,jquery,html,opera,opera-mini"
1,store blob data with django and sqlite first o...,"python,django,orm,blob,blobstorage"
2,classes and functions in js? here is some samp...,"javascript,class"
3,serving media in django i was going through se...,"python,django,django-templates"
4,which for loop is more efficient in javascript...,"javascript,for-loop,comparison,performance"
5,parameters in ajax php callback function i am ...,"php,javascript,jquery,ajax"
6,can one author a library api – one accessible ...,"python,c++,c,api,cython"
7,auto convert dictionary into list of dictionar...,"python,list,dictionary"
8,complex numbers and fractals i am trying to ad...,"python,c,complex-numbers,complextype,fractals"
9,"""current instrumentation disallows attach"" ins...","ipad,ios6,mobile-safari,xcode4.5"


# After filtering and writing dataframe to CSV, upload to **Google CLoud Storage** (GCS)
## the second cell runs a shell command. 
- use the "!" charector to initiate shell command from **jupitor notebook**

In [320]:
!gsutil cp file_write.csv gs://[BUCKET-NAME]/

Copying file://file_write.csv [Content-Type=text/csv]...
\ [1 files][ 36.1 MiB/ 36.1 MiB]                                                
Operation completed over 1 objects/36.1 MiB.                                     


# Creating a model, training and deploying
## created based off of this resourece:https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl-text-classification.ipynb

In [ ]:
#pip install required libraries
! pip install {USER_FLAG} --upgrade google-cloud-aiplatform google-cloud-storage jsonlines

In [333]:
from google.cloud import aiplatform, storage, bigquery
from google.protobuf import json_format

REGION = "[REGION]"
PROJECT_ID = "[PROJECT_ID]"
BUCKET_NAME = "[BUCKET_NAME]"
FILE_NAME = "[FILE_NAME]"

src_uris ="gs://{}/{}".format(BUCKET_NAME, FILE_NAME)
display_name = "[DISPLAY_NAME -- your choice of naming"

aiplatform.init(project=PROJECT_ID, location=REGION)

## create text multi label dataset

In [334]:
#creating text dataset
ds = aiplatform.TextDataset.create(
    display_name=display_name,
    gcs_source=src_uris,
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.multi_label_classification,
    sync=True,
)

INFO:google.cloud.aiplatform.datasets.dataset:Creating TextDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TextDataset backing LRO: projects/513072429921/locations/us-central1/datasets/7070712987622834176/operations/6270467946523394048
INFO:google.cloud.aiplatform.datasets.dataset:TextDataset created. Resource name: projects/513072429921/locations/us-central1/datasets/7070712987622834176
INFO:google.cloud.aiplatform.datasets.dataset:To use this TextDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TextDataset('projects/513072429921/locations/us-central1/datasets/7070712987622834176')
INFO:google.cloud.aiplatform.datasets.dataset:Importing TextDataset data: projects/513072429921/locations/us-central1/datasets/7070712987622834176
INFO:google.cloud.aiplatform.datasets.dataset:Import TextDataset data backing LRO: projects/513072429921/locations/us-central1/datasets/7070712987622834176/operations/1057551377842044928
INFO:google.cloud.aipl

## pull dataset id of most recent dataset created

In [335]:
dataset_id = ds.resource_name.split("/")[-1]
text_dataset = aiplatform.TextDataset(dataset_id)

## create training job description

In [ ]:
# Define the training job
training_job_display_name = "stack_question_training" # change to your liking
job = aiplatform.AutoMLTextTrainingJob(
    display_name=training_job_display_name,
    prediction_type="classification",
    multi_label=True,
)

## run model training
- this took about 4 hours in this test

In [ ]:
model_display_name = "stack_question_model" # change to your liking

# Run the training job
model = job.run(
    dataset=text_dataset,
    model_display_name=model_display_name,
    training_fraction_split=0.7,
    validation_fraction_split=0.2,
    test_fraction_split=0.1,
    sync=True,
)

##  deploy model

In [ ]:
deployed_model_display_name = "model-endpoint-stack-questions" # change to your liking

endpoint = model.deploy(
    deployed_model_display_name=deployed_model_display_name, sync=True
)

# once model is deployed: 
- find model endpoint id through vertex ai "endpoints" page on **Google Cloud Console**
- example: "[GOOGLE_CLOUD_CONSOLE]/vertex-ai/endpoints?project=[YOUR PROJECT HERE]"
- funciton below is pulled from: https://github.com/googleapis/python-aiplatform/blob/main/samples/snippets/prediction_service/predict_text_classification_single_label_sample.py
- **this function** will test your deployed model and is an example of how to use your model to recieve predictions

## visualize performance of your model
- navigate to "model" page of your vertex ai dashboard
- here you can inspect the performance of each of your models


In [ ]:
def predict_text_classification_multi_lable(
    project: str,
    endpoint_id: str,
    content: str,
    location: str ,
    api_endpoint: str = "[your model location]-aiplatform.googleapis.com",
):
    api_endpoint = location + "-aiplatform.googleapis.com"
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    instance = predict.instance.TextClassificationPredictionInstance(
        content=content,
    ).to_value()
    instances = [instance]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # See gs://google-cloud-aiplatform/schema/predict/prediction/text_classification.yaml for the format of the predictions.
    predictions = response.predictions
    for prediction in predictions:
        dictionary = dict(prediction)
        print(" prediction:")
        for index in range(len(dictionary['displayNames'])):
            print("\t", dictionary['displayNames'][index], ": " , dictionary['confidences'][index])
    return dictionary 
            

In [ ]:
results =  predict_text_classification_multi_lable(
            "[PROJECT_ID]",
            endpoint_id="[YOUR ENDPOINT ID]",
            location="[LOCATION FOR YOUR ENDPOINT]",
            content= "CONTENT YOU WANT TO TEST"
        ) 